In [1]:
#
# Simple example of a Julia linear pogramming model
#
# Convert from notebook to julia program:
#   python3 nb_to_jl.py lp_claude_revised_simplex.ipynb
#
# Run the program:
#   julia lp_llama_ipm.jl -f ../benchmarks/mps_files/ex97.mps -v --no_presolve
#
 
using LinearAlgebra
using SparseArrays
using Random
using ArgParse

# use this for command line version
const DEFAULT_USE_SIMPLEX_METHOD = true
const DEFAULT_USE_INTERIOR_POINT_METHOD = false
const DEFAULT_USE_NO_PRESOLVE = true 
# const DEFAULT_USE_SPARSE_MATRIX = false
const DEFAULT_MINIMIZE_OBJECTIVE = true
const DEFAULT_MAXIMIZE_OBJECTIVE = false
const DEFAULT_VERBOSE = false

# const DEFAULT_USE_SIMPLEX_METHOD = true
# const DEFAULT_USE_INTERIOR_POINT_METHOD = false
# const DEFAULT_USE_NO_PRESOLVE = false 
# # const DEFAULT_USE_SPARSE_MATRIX = false
# const DEFAULT_MINIMIZE_OBJECTIVE = true
# const DEFAULT_MAXIMIZE_OBJECTIVE = false
# const DEFAULT_VERBOSE = true

global options = Dict()

const COMMAND_LINE_OPTION_FILENAME = "filename"
const COMMAND_LINE_OPTION_USE_NO_PRESOLVE = "no_presolve"
# const COMMAND_LINE_OPTION_USE_SPARSE_MATRIX = "sparse"
const COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD = "simplex"
const COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD = "interior"
const COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE = "min"
const COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE = "max"
const COMMAND_LINE_OPTION_VERBOSE = "verbose"

const OPTION_FILENAME = "filename"
const OPTION_USE_SIMPLEX_METHOD = "simplex"
const OPTION_USE_NO_PRESOLVE = "use_no_presolve"
# const OPTION_USE_SPARSE_MATRIX = "use_sparse_matrix"
const OPTION_USE_INTERIOR_POINT_METHOD = "ipm"
const OPTION_MINIMIZE_OBJECTIVE = "minimize"
const OPTION_MAXIMIZE_OBJECTIVE = "maximize"
const OPTION_VERBOSE = "verbose"

# https://en.wikipedia.org/wiki/Revised_simplex_method
const MPS_EXAMPLE::String = "../benchmarks/mps_files/ex4-3.mps"
# const MPS_EXAMPLE::String = """
# NAME          APPLIED_INTEGER_PROGRAMMING_9_7

# OBJSENSE
#  MAX

# ROWS
#  N  OBJ
#  L  ROW1
#  L  ROW2
#  L  ROW3

# COLUMNS
#     X1        OBJ       4
#     X1        ROW1      1
#     X1        ROW2      2
#     X1        ROW3     -3
#     X2        OBJ       3
#     X2        ROW1      2
#     X2        ROW2     -1
#     X2        ROW3      2
#     X3        OBJ       1
#     X3        ROW1      3
#     X3        ROW2      2
#     X3        ROW3      1
#     X4        OBJ       7
#     X4        ROW1      1
#     X4        ROW2      2
#     X4        ROW3     -1
#     X5        OBJ       6
#     X5        ROW1     -3
#     X5        ROW2      1
#     X5        ROW3      2

# RHS
#     RHS1      ROW1      9
#     RHS1      ROW2     10
#     RHS1      ROW3     11

# BOUNDS
#  LO BOUND1    X1        0
#  LO BOUND1    X2        0
#  LO BOUND1    X3        0
#  LO BOUND1    X4        0
#  LO BOUND1    X5        0

# ENDATA
# """
#=
/opt/homebrew/Cellar/highs/1.7.2/bin/highs --solution_file ex97_highs.txt ex97.mps
Running HiGHS 1.7.2 (git hash: n/a): Copyright (c) 2024 HiGHS under MIT licence terms
LP   ex97 has 3 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 7e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 1e+01]
Presolving model
3 rows, 5 cols, 15 nonzeros  0s
3 rows, 5 cols, 15 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 5(-0); elements 15(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999978563e+01 Ph1: 3(11); Du: 5(21) 0s
          4     9.4000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  9.4000000000e+01
HiGHS run time      :          0.00

Model status
Optimal

# Primal solution values
Feasible
Objective 94
# Columns 5
X1 7
X2 10
X3 0
X4 0
X5 6
# Rows 3
ROW1 9
ROW2 10
ROW3 11

# Dual solution values
Feasible
# Columns 5
X1 0
X2 0
X3 -16.4285714285714
X4 -2.2380952380952
X5 0
# Rows 3
ROW1 1.4761904761905
ROW2 5.1904761904762
ROW3 2.6190476190476

# Basis
HiGHS v1
Valid
# Columns 5
1 1 0 0 1 
# Rows 3
2 2 2 
=#

"../benchmarks/mps_files/ex4-3.mps"

In [2]:
function is_running_in_notebook()
    # Check if running in VS Code's Jupyter notebook environment
    if (haskey(ENV, "VSCODE_PID") || haskey(ENV, "VSCODE_CWD"))
        return true
    # Check if running in a general Jupyter environment (including VS Code)
    elseif isdefined(Main, :IJulia) && Main.IJulia.inited
        return true
    else
        return false
    end
end

is_running_in_notebook (generic function with 1 method)

In [3]:
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--filename", "-f"
            help = "path to the problem file (mps format)"
            arg_type = String
            required = true
        "--interior", "-i"
            help = "use interior point method (LP only)"
            action = :store_true
        "--min"
            help = "minimization of the objective function (default)"
            action = :store_true            
        "--max"
            help = "maximization of the objective function"
            action = :store_true            
        "--no_presolve"
            help = "do not presolve (default false)"
            action = :store_true
        "--simplex", "-s"
            help = "use simplex method (default)"
            action = :store_true          
        # "--sparse"
        #     help = "use sparce matrix representation"
        #     action = :store_true 
        "--verbose", "-v"
            help = "verbose output"
            action = :store_true
    end

    return parse_args(s)
end

parse_commandline (generic function with 1 method)

In [4]:
# Define a struct to represent a Linear Programming problem
struct LPProblem
    is_minimize::Bool  # True if the objective is to minimize
    c::Vector{Float64}  # Objective function coefficients
    A::Matrix{Float64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    l::Vector{Float64} # variable lower bounds
    u::Vector{Float64} # variable upper bounds
    vars::Vector{String}  # Variable names
    constraint_types::Vector{Char}  # Constraint types
end

In [5]:
include("read_mps.jl")
using .MPSReader


# file_path = "../benchmarks/mps_files/ex4-3.mps"

# file_string =  read_mps_from_file(file_path)


# function read_mps_from_string(mps_string::String)
#     lines = split(mps_string, '\n')
#     sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => Dict(), "RHS" => Dict(), "BOUNDS" => Dict())
#     current_section = ""
#     objective_name = ""
#     is_minimize = options[OPTION_MINIMIZE_OBJECTIVE] ? true : false

#     objective_set = false
#     for line in lines
#         words = split(line)
#         (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

#         if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
#             current_section = words[1]
#             continue
#         end

#         if current_section == "NAME"
#             sections["NAME"] = words[1]
#         elseif current_section == "OBJSENSE"
#             if words[1] == "MAX"
#                 is_minimize = false
#                 objective_set = true
#             elseif words[1] == "MIN"
#                 is_minimize = true
#                 objective_set = true
#             end
#         elseif current_section == "ROWS"
#             row_type, row_name = words
#             push!(sections["ROWS"], (type=row_type, name=row_name))
#             if row_type == "N"
#                 objective_name = row_name
#             end
#         elseif current_section == "COLUMNS"
#             col_name, row_name, value = words
#             value = parse(Float64, value)
#             if !haskey(sections["COLUMNS"], col_name)
#                 sections["COLUMNS"][col_name] = Dict()
#             end
#             sections["COLUMNS"][col_name][row_name] = value
#         elseif current_section == "RHS"
#             if length(words) == 3
#                 _, row_name, value = words
#             else
#                 row_name, value = words[2:3]
#             end
#             sections["RHS"][row_name] = parse(Float64, value)
#         elseif current_section == "BOUNDS"
#             bound_type, _, var_name, value = words
#             if !haskey(sections["BOUNDS"], var_name)
#                 sections["BOUNDS"][var_name] = Dict()
#             end
#             sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
#         end
#     end

#     # Convert to LPProblem structure
#     vars = collect(keys(sections["COLUMNS"]))
#     n_vars = length(vars)
#     n_constraints = count(row -> row.type != "N", sections["ROWS"])

#     c = zeros(n_vars)
#     A = zeros(n_constraints, n_vars)
#     b = zeros(n_constraints)
#     constraint_types = Char[]

#     # Populate objective function
#     for (i, var) in enumerate(vars)
#         if haskey(sections["COLUMNS"][var], objective_name)
#             c[i] = sections["COLUMNS"][var][objective_name]
#         end
#     end

#     # Populate constraint matrix and right-hand side
#     constraint_index = 0
#     for row in sections["ROWS"]
#         if row.type != "N"
#             constraint_index += 1
#             push!(constraint_types, row.type[1])  # Store constraint type
#             for (i, var) in enumerate(vars)
#                 if haskey(sections["COLUMNS"][var], row.name)
#                     A[constraint_index, i] = sections["COLUMNS"][var][row.name]
#                 end
#             end
#             b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
#             # Adjust for 'G' type constraints
#             if row.type == "G"
#                 A[constraint_index, :] *= -1
#                 b[constraint_index] *= -1
#             end
#         end
#     end

#     # Process bound constraints
#     lb = fill(-Inf, n_vars)
#     ub = fill(Inf, n_vars)
#     for (i, var) in enumerate(vars)
#         if haskey(sections["BOUNDS"], var)
#             bounds = sections["BOUNDS"][var]
#             if haskey(bounds, "LO")
#                 lb[i] = bounds["LO"]
#             end
#             if haskey(bounds, "UP")
#                 ub[i] = bounds["UP"]
#             end
#             if haskey(bounds, "FX")
#                 lb[i] = ub[i] = bounds["FX"]
#             end
#         else
#             lb[i] = 0.0  # Default lower bound is 0 if not specified
#         end
#     end

#     if options[OPTION_VERBOSE] && objective_set
#         println("\nWARNING: Objective function is set to $(is_minimize ? OPTION_MINIMIZE_OBJECTIVE : OPTION_MAXIMIZE_OBJECTIVE) in the MPS file overriding any command line option\n")
#     end

#     return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
# end

In [6]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, sparse(lp.A), lp.b
    l, u = copy(lp.l), copy(lp.u)  # Create copies to avoid modifying the original bounds
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] ≈ A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] ≈ -A[j, :] && abs(b[i] + b[j]) < eps &&
                    ((constraint_types[i] == 'L' && constraint_types[j] == 'G') ||
                     (constraint_types[i] == 'G' && constraint_types[j] == 'L'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) ≈ 1 && constraint_types[i] == 'E'
                val = b[i] / col[i]
                if l[j] <= val && val <= u[j]
                    fixed_vars[vars[j]] = val
                    keep_cols[j] = false
                    b .-= val * col
                end
            end
        end
    end

    # Conservative bound tightening with feasibility checks
    for j in 1:n
        lower_bounds = Float64[]
        upper_bounds = Float64[]
        
        println("Analyzing bounds for variable $(vars[j]):")
        println("  Initial bounds: [$(l[j]), $(u[j])]")
        
        for i in 1:m
            if A[i,j] == 0
                continue
            end
            
            new_bound = b[i] / A[i,j]
            println("  Constraint $(i): A[i,j] = $(A[i,j]), b[i] = $(b[i]), new_bound = $new_bound")
            
            if constraint_types[i] == 'L'
                if A[i,j] > 0
                    push!(upper_bounds, new_bound)
                else
                    push!(lower_bounds, new_bound)
                end
            elseif constraint_types[i] == 'G'
                if A[i,j] > 0
                    push!(lower_bounds, new_bound)
                else
                    push!(upper_bounds, new_bound)
                end
            elseif constraint_types[i] == 'E'
                push!(lower_bounds, new_bound)
                push!(upper_bounds, new_bound)
            end
        end
        
        println("  Potential lower bounds: $lower_bounds")
        println("  Potential upper bounds: $upper_bounds")
        
        if !isempty(lower_bounds)
            new_lower = maximum(lower_bounds)
            if new_lower > l[j] + eps
                println("  Tightening lower bound: $new_lower (prev: $(l[j]))")
                l[j] = new_lower
            end
        end
        
        if !isempty(upper_bounds)
            new_upper = minimum(upper_bounds)
            if new_upper < u[j] - eps
                println("  Tightening upper bound: $new_upper (prev: $(u[j]))")
                u[j] = new_upper
            end
        end
        
        # Ensure lower bound doesn't exceed upper bound
        if l[j] > u[j] + eps
            println("  WARNING: Lower bound ($(l[j])) exceeds upper bound ($(u[j])) for $(vars[j])")
            println("  Adjusting bounds to maintain feasibility")
            avg_bound = (l[j] + u[j]) / 2
            l[j] = avg_bound - eps
            u[j] = avg_bound + eps
        end
        
        println("  Final bounds for $(vars[j]): [$(l[j]), $(u[j])]")
        println()
    end

    # Ensure bounds are consistent with the objective
    for j in 1:n
        if !is_minimize && u[j] < c[j]
            println("Adjusting upper bound for $(vars[j]) to match objective: $(c[j]) (prev: $(u[j]))")
            u[j] = c[j]
        elseif is_minimize && l[j] > c[j]
            println("Adjusting lower bound for $(vars[j]) to match objective: $(c[j]) (prev: $(l[j]))")
            l[j] = c[j]
        end
    end
   
    # Ensure bounds are consistent with constraints
    for j in 1:n
        upper_bounds = [b[i] / A[i, j] for i in 1:m if A[i, j] > 0 && constraint_types[i] == 'L']
        lower_bounds = [b[i] / A[i, j] for i in 1:m if A[i, j] < 0 && constraint_types[i] == 'L']
        
        if !isempty(upper_bounds)
            max_possible_value = minimum(upper_bounds)
            if u[j] > max_possible_value
                println("Adjusting upper bound for $(vars[j]) based on constraints: $max_possible_value (prev: $(u[j]))")
                u[j] = max_possible_value
            end
        end
        
        if !isempty(lower_bounds)
            min_possible_value = maximum(lower_bounds)
            if l[j] < min_possible_value
                println("Adjusting lower bound for $(vars[j]) based on constraints: $min_possible_value (prev: $(l[j]))")
                l[j] = min_possible_value
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    l_new = l[keep_cols]
    u_new = u[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = 0.0
    for (j, var) in enumerate(vars)
        if haskey(fixed_vars, var)
            obj_adjust += c[j] * fixed_vars[var]
        end
    end

    if options[OPTION_VERBOSE]
        println("\nPresolve summary:")
        println("  Original problem size: $(m) x $(n)")
        println("  Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("  Number of fixed variables: $(length(fixed_vars))")
        println("  Objective adjustment: $obj_adjust")
        println("  Updated bounds:")
        for (var, lb, ub) in zip(vars_new, l_new, u_new)
            println("    $var: [$lb, $ub]")
        end
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, l_new, u_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

presolve (generic function with 1 method)

In [7]:
function convert_to_standard_form(lp::LPProblem)
    c, A, b, l, u = lp.c, lp.A, lp.b, lp.l, lp.u
    m, n = size(A)
    
    # Handle lower and upper bounds
    for i in 1:n
        if l[i] > -Inf
            A = [A; zeros(1, n)]
            A[end, i] = -1
            push!(b, -l[i])
            push!(lp.constraint_types, 'L')
        end
        if u[i] < Inf
            A = [A; zeros(1, n)]
            A[end, i] = 1
            push!(b, u[i])
            push!(lp.constraint_types, 'L')
        end
    end
    
    m, n = size(A)
    new_A = similar(A, m, n + m)
    new_b = copy(b)
    new_c = [c; zeros(m)]
    
    for i in 1:m
        if lp.constraint_types[i] == 'L'
            new_A[i, :] = [A[i, :]; zeros(i-1); 1; zeros(m-i)]
        elseif lp.constraint_types[i] == 'G'
            new_A[i, :] = [-A[i, :]; zeros(i-1); 1; zeros(m-i)]
            new_b[i] = -new_b[i]
        elseif lp.constraint_types[i] == 'E'
            new_A[i, :] = [A[i, :]; zeros(m)]
        end
    end
    
    if !lp.is_minimize
        new_c = -new_c
    end
    
    return new_A, new_b, new_c
end

convert_to_standard_form (generic function with 1 method)

Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.

In [8]:
function revised_simplex(lp::LPProblem)
    println("Converting problem to standard form...")
    A, b, c = convert_to_standard_form(lp)
    m, n = size(A)
    
    println("\nStandard form problem:")
    println("  Objective function coefficients c: ", c)
    println("  Constraint matrix A: ", A)
    println("  Right-hand side b: ", b)
    println("  Variables: ", [lp.vars; ["s$i" for i in 1:(n-length(lp.vars))]])
    println("  Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    println("  constraint_types = $(lp.constraint_types)")
    
    # Initialize basis with slack variables
    B = collect((length(lp.vars)+1):n)
    N = collect(1:length(lp.vars))
    
    println("\nInitial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]
        x_B = B_matrix \ b
        println("  Basic solution x_B: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'
        c_N = c[N] - A[:, N]' * y
        println("  Dual variables y: ", y)
        println("  Reduced costs c_N: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .>= -1e-10)
            x = zeros(n)
            x[B] = x_B
            println("\nOptimal solution found:")
            obj_value = dot(lp.is_minimize ? c : -c, x)
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable
        e = argmin(c_N)
        q = N[e]
        println("  Entering variable: ", q)
        
        # Step 5: Compute direction
        d = B_matrix \ A[:, q]
        println("  Direction d: ", d)
        
        # Step 6 & 7: Check unboundedness and choose leaving variable
        if all(d .<= 1e-10)
            error("Problem is unbounded")
        end
        
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf
        valid_ratios = filter(x -> x > 0, ratios)
        if isempty(valid_ratios)
            error("Problem is unbounded")
        end
        l = argmin(valid_ratios)
        p = B[l]
        println("  Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        println("  New basis: ", B)
        println("  New non-basic variables: ", N)
        
        if iteration > 10  # FIXME: Add a better termination criterion
            error("Maximum iterations reached")
        end
    end
end

revised_simplex (generic function with 1 method)

In [9]:
function corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
    println("Computing corrector direction:")
    
    # Get dimensions
    m, n = size(A)
    println("  m: $m, n: $n")

    # Dimension checks
    if size(A, 1) != m || size(A, 2) != n
        error("A should be an m×n matrix. Got size(A) = $(size(A))")
    end
    if length(r_b) != m || length(r_s) != m || length(s) != m || length(Δy_aff) != m
        error("r_b, r_s, s, and Δy_aff should be m-dimensional vectors. 
               Got lengths: r_b ($(length(r_b))), r_s ($(length(r_s))), 
               s ($(length(s))), Δy_aff ($(length(Δy_aff)))")
    end
    if length(x) != n || length(Δx_aff) != n
        error("x and Δx_aff should be n-dimensional vectors. 
               Got lengths: x ($(length(x))), Δx_aff ($(length(Δx_aff)))")
    end

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    Δx_aff = vec(Δx_aff)
    Δy_aff = vec(Δy_aff)

    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")
    println("  Δx_aff: $Δx_aff")
    println("  Δy_aff: $Δy_aff")

    # Compute Δs_aff
    Δs_aff = r_s - A * Δx_aff

    # Compute μ (barrier parameter)
    μ = (dot(x, A' * s) / n) * ((s + Δs_aff)' * (A * (x + Δx_aff)) / (s' * (A * x)))^3

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b - A * Δx_aff;
           A' * r_s - A' * Δs_aff - (x .* (A' * Δs_aff) + A' * s .* Δx_aff - μ * ones(n))]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy_cor = direction[1:m]
    Δx_cor = direction[m+1:end]

    return Δx_cor, Δy_cor
end

corrector_direction (generic function with 1 method)

In [10]:
function affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
    println("Computing affine scaling direction:")
    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")

    m, n = size(A)
    println("  m: $m, n: $n")

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    y = vec(y)
    r_c = vec(r_c)

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b; -(A' * y + r_c)]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy = direction[1:m]
    Δx = direction[m+1:end]

    return Δx, Δy
end

affine_scaling_direction (generic function with 1 method)

In [11]:
function compute_step_length(v, Δv)
    α = 1.0
    for i in eachindex(v)
        if Δv[i] < 0
            α = min(α, -v[i] / Δv[i])
        end
    end
    return α
end

compute_step_length (generic function with 1 method)

In [12]:
#=
The rest of the Interior Point Method implementation looks correct. The main loop in the interior_point_method function follows the standard steps:

Compute residuals
Compute affine scaling direction
Compute corrector direction
Combine directions and update variables
Update barrier parameter
Check for convergence

The method also includes proper logging and debugging output, which is helpful for understanding the algorithm's progress.
To further improve the code, you might consider:

Adding error handling for cases where the problem doesn't converge within the maximum number of iterations.
Implementing a dynamic update strategy for the barrier parameter μ.
Adding more sophisticated step length calculations to ensure variables remain positive.
=#


function interior_point_method(lp::LPProblem; 
                               tolerance=1e-8, 
                               max_iterations=1000, 
                               barrier_parameter=0.1)
    # Initialize variables
    n = length(lp.vars)
    m = size(lp.A, 1)
    x = ones(n)
    s = ones(m)
    y = zeros(m)
    μ = barrier_parameter
    iteration = 0

    println("Starting IPM solver with parameters:")
    println("  Tolerance: $tolerance")
    println("  Max iterations: $max_iterations")
    println("  Barrier parameter: $barrier_parameter")

    # Main loop
    while iteration < max_iterations
        println("\nIteration $iteration:")

        # Compute residuals
        r_c = lp.c - A' * y
        r_b = b - A * x
        r_s = s - (A * x - b)
        println("  Residuals:")
        println("    r_c: $r_c")
        println("    r_b: $r_b")
        println("    r_s: $r_s")

        # Compute affine scaling direction
        Δx_aff, Δy_aff = affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
        Δs_aff = r_s - A * Δx_aff
        println("  Affine scaling direction:")
        println("    Δx_aff: $Δx_aff")
        println("    Δs_aff: $Δs_aff")
        println("    Δy_aff: $Δy_aff")

        # Compute corrector direction
        Δx_cor, Δy_cor = corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
        Δs_cor = r_s - A * Δx_cor
        println("  Corrector direction:")
        println("    Δx_cor: $Δx_cor")
        println("    Δs_cor: $Δs_cor")
        println("    Δy_cor: $Δy_cor")

        # Combine directions and update variables
        Δx = Δx_aff + Δx_cor
        Δs = Δs_aff + Δs_cor
        Δy = Δy_aff + Δy_cor

        # Compute step length
        α_pri = compute_step_length(x, Δx)
        α_dual = compute_step_length(s, Δs)
        α = min(0.99 * min(α_pri, α_dual), 1)

        x += α * Δx
        s += α * Δs
        y += α * Δy
        println("  Updated variables:")
        println("    x: $x")
        println("    s: $s")
        println("    y: $y")

        # Update barrier parameter
        μ = (dot(x, A' * s)) / n
        println("  Updated barrier parameter: $μ")

        # Check convergence
        if norm([r_c; r_b; r_s]) < tolerance && μ < tolerance
            println("Converged in $iteration iterations")
            break
        end

        iteration += 1
    end

    return x, y, iteration
end

interior_point_method (generic function with 1 method)

In [13]:
# main program

if is_running_in_notebook()
    println("Running as a notebook")    
else
    println("Running as a script")
end

options[OPTION_USE_SIMPLEX_METHOD] = DEFAULT_USE_SIMPLEX_METHOD
options[OPTION_USE_NO_PRESOLVE] = DEFAULT_USE_NO_PRESOLVE
# options[OPTION_USE_SPARSE_MATRIX] = DEFAULT_USE_SPARSE_MATRIX
options[OPTION_USE_INTERIOR_POINT_METHOD] = DEFAULT_USE_INTERIOR_POINT_METHOD
options[OPTION_MINIMIZE_OBJECTIVE] = DEFAULT_MINIMIZE_OBJECTIVE
options[OPTION_MAXIMIZE_OBJECTIVE] = DEFAULT_MAXIMIZE_OBJECTIVE
options[OPTION_VERBOSE] = DEFAULT_VERBOSE

mps_string = MPS_EXAMPLE
if !is_running_in_notebook()
    parsed_args = parse_commandline()
    # println("Parsed args: $parsed_args")

    # Process command-line name
    if haskey(parsed_args, COMMAND_LINE_OPTION_FILENAME) && !isnothing(parsed_args[COMMAND_LINE_OPTION_FILENAME])
        options[OPTION_FILENAME] = parsed_args[COMMAND_LINE_OPTION_FILENAME]
        if isfile(options[OPTION_FILENAME])
            mps_string = String(read(options[OPTION_FILENAME]))
        else            
            error("File not found or could not read: '$(options[OPTION_FILENAME])'")
        end
    end

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD])
        options[OPTION_USE_SIMPLEX_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end


    if haskey(parsed_args, COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE])
        options[OPTION_MINIMIZE_OBJECTIVE] = parsed_args[COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE])
        options[OPTION_MAXIMIZE_OBJECTIVE] = parsed_args[COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE]
    end

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_NO_PRESOLVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_NO_PRESOLVE])
        options[OPTION_USE_NO_PRESOLVE] = parsed_args[COMMAND_LINE_OPTION_USE_NO_PRESOLVE]
    end
    # if haskey(parsed_args, COMMAND_LINE_OPTION_USE_SPARSE_MATRIX) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX])
    #     options[OPTION_USE_SPARSE_MATRIX] = parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX]
    # end
    if haskey(parsed_args, COMMAND_LINE_OPTION_VERBOSE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_VERBOSE])
        options[OPTION_VERBOSE] = parsed_args[COMMAND_LINE_OPTION_VERBOSE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end
end

# set defaults if not explicably set
if !options[OPTION_USE_SIMPLEX_METHOD] && !options[OPTION_USE_INTERIOR_POINT_METHOD]
    options[OPTION_USE_SIMPLEX_METHOD] = true
elseif options[OPTION_USE_SIMPLEX_METHOD] && options[OPTION_USE_INTERIOR_POINT_METHOD]
    error("Cannot use both simplex and interior point methods")
end

if !options[OPTION_MINIMIZE_OBJECTIVE] && !options[OPTION_MAXIMIZE_OBJECTIVE]
    options[OPTION_MINIMIZE_OBJECTIVE] = true
elseif options[OPTION_MINIMIZE_OBJECTIVE] && options[OPTION_MAXIMIZE_OBJECTIVE]
    error("Cannot maximize and minimize the objective at the same time")
end


if options[OPTION_VERBOSE]
    println("Options:")
    println("  Use no presolve: $(options[OPTION_USE_NO_PRESOLVE])")
    # println("  Use sparse matrix: $(options[OPTION_USE_SPARSE_MATRIX])")
    println("  Use simplex method: $(options[OPTION_USE_SIMPLEX_METHOD])")
    println("  Use interior point method: $(options[OPTION_USE_INTERIOR_POINT_METHOD])")
    println("  Objective: $(options[OPTION_MINIMIZE_OBJECTIVE] ? OPTION_MINIMIZE_OBJECTIVE : OPTION_MAXIMIZE_OBJECTIVE)")
    println("  Verbose output: $(options[OPTION_VERBOSE])")
end

# Example usage
lp = read_mps_from_file(MPS_EXAMPLE)


if options[OPTION_VERBOSE]
    println("\nOriginal problem:")
    println("  Minimize: $(lp.is_minimize)")
    println("  c = $(lp.c)")
    println("  A = $(lp.A)")
    println("  b = $(lp.b)")
    println("  l = $(lp.l)")
    println("  u = $(lp.u)")        
    println("  vars = $(lp.vars)")
    println("  constraint_types = $(lp.constraint_types)")
end

# Presolve the problem
if !options[OPTION_USE_NO_PRESOLVE]
    println("\nPresolving the problem")
    lp, fixed_vars, obj_adjust = presolve(lp)

    println("\nFixed variables:")
    for (var, val) in fixed_vars
        println("$var = $val")
    end

    println("\nReduced problem:")
    println("  Minimize: $(lp.is_minimize)")
    println("  c = $(lp.c)")
    println("  A = $(lp.A)")
    println("  b = $(lp.b)")
    println("  l = $(lp.l)")
    println("  u = $(lp.u)")     
    println("  vars = $(lp.vars)")
    println("  constraint_types = $(lp.constraint_types)")
end

# solve the LP problem

if options[OPTION_USE_INTERIOR_POINT_METHOD]
        println("\nSolving the problem using the Interior Point Method")
        execution_time = @elapsed x, y, iterations = interior_point_method(lp) 
else
    # if options[OPTION_USE_SPARSE_MATRIX]
    #     println("\nSolving the problem using the Revised Simplex Method with sparse matrices")
    #     execution_time = @elapsed x, obj_value = revised_simplex_sparse(lp)
    # else
        println("\nSolving the problem using the Revised Simplex Method with dense matrices")
        execution_time = @elapsed x, obj_value = revised_simplex(lp)
    # end
end
println("\nExecution time: $execution_time seconds")

# show the solution

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")

# test problem
# is_minimize = false
# c = [4.0, 3.0, 1.0, 7.0, 6.0]
# A = [1.0 2.0 3.0 1.0 -3.0;
#      2.0 -1.0  2.0  2.0  1.0;
#     -3.0  2.0  1.0  -1.0  2.0]
# b = [9.0, 10.0, 11.0]
# vars = ["x1", "x2", "x3", "x4", "x5"]
# constraint_types = ['L', 'L', 'L']
# l = [0.0, 0.0, 0.0, 0.0, 0.0]
# u = [Inf, Inf, Inf, Inf, Inf]

# lp = LPProblem(is_minimize, c, A, b, l, u, vars, constraint_types)

# # Solve the LP problem using the Revised Simplex Method
# x, obj_value = revised_simplex(lp)

# println("\nFinal solution:")
# for (var, val) in zip(lp.vars, x)
#     println("$var = $(round(val, digits=6))")
# end
# println("Objective value: $(round(obj_value, digits=6))")

# Final solution:
# X3 = 0.0
# X4 = 0.0
# X5 = 6.0
# X2 = 10.0
# X1 = 7.0
# Objective value: 94.0

Running as a notebook

Solving the problem using the Revised Simplex Method with dense matrices


MethodError: MethodError: no method matching revised_simplex(::Main.MPSReader.lpProblem.LPProblem)

Closest candidates are:
  revised_simplex(!Matched::LPProblem)
   @ Main ~/Desktop/lp_code/lp_code/lp_julia/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:1
